# cascading arbitrary circuit elements

always restart notebook to clear `dsdobject` singletons!

In [1]:
import sys; sys.path.append("../src")

from compile import *
from utils import cfg_logging_handler

handler = cfg_logging_handler()

## weighted sum

In [2]:
import numpy as np

# class memory = column
W = np.array([
    [0.8, 0.2],
    [0.1, 0.4],
    [0.1, 0.0],
])
# column sum <= 1 required

W.shape

(3, 2)

**column = memory**

In [3]:
circ = DSDCircuit(fuel_multiplier=1.0)

patt_size, n_classes = W.shape

inp = circ.new_signal("bi", patt_size)

prod = circ.new_signal("p", n_classes)
th_ext = circ.add_global(TH_EXT_NAME, 2)

wsum = circ.new_signal("s", n_classes)

def g(i: int, module: WeightMultiplication) -> Strand:
    # assuming summation layer is next
    return strand3to5([module.output.domain(i), th_ext])

modules = [
    WeightMultiplication(inp, prod, W, out_strand=g),
    Summation(prod, wsum),
]
circ.add_modules(modules, add_reporting=True)
circ.export_PIL(output_file=f"../artifacts/weighted-sum/{patt_size}x{n_classes}.pil")

INFO - Exported PIL to ..\artifacts\weighted-sum\3x2.pil


### simplified (only multiplication)

In [7]:
patt_size, n_classes = W.shape
circ = DSDCircuit(fuel_multiplier=1.0)

inp = circ.new_signal("bi", patt_size)
prod = circ.new_signal("p", n_classes)

mul = WeightMultiplication(inp, prod, W,
                           out_strand=lambda i, module: [module.output.domain(i)]) # no extension (reporting not designed to support it)

circ.add_modules([mul], add_reporting=True)

## WTA

In [2]:

circ = DSDCircuit(fuel_multiplier=2.0)
n_classes = 2

restoration_th_len = 5
th_ext_len = 5
# make sure the sum is below recognition domain length!

comp = circ.new_signal("c", n_classes, toehold_len=restoration_th_len)
th_ext = circ.add_global(TH_EXT_NAME, th_ext_len)

out = circ.new_signal("bo", n_classes)

activation = [
    PairwiseAnnihilation(comp, annihilator_conc=1.5),
    SignalRestoration(comp, out),
]

circ.add_modules(activation, add_reporting=True)

circ.export_PIL(standard_conc=50,output_file=
                f"../artifacts/wta-circuit/{n_classes}-WTA(T={restoration_th_len},s={th_ext_len}).pil")

INFO - Exported PIL to ..\artifacts\wta-circuit\2-WTA(T=5,s=5).pil


## Signal reversal

In [2]:
sigrev = DSDCircuit(fuel_multiplier=2.0)
n_classes = 3

inp = sigrev.new_signal("x", n_classes, LTA_PAPER_DOMAIN_LEN["input_signal"])
sigrev.add_global(TH_EXT_NAME, 2)
out = sigrev.new_signal("y", n_classes, LTA_PAPER_DOMAIN_LEN["reversed_signal"])

modules = [
    SignalReversal(inp, out, reversible_design=False, reversal_gate_conc=1/(n_classes - 1)),
    Reporting(out, reporter_conc=2)
]

sigrev.add_modules(modules, add_reporting=False)
sigrev.export_PIL(standard_conc=50, output_file=f"../artifacts/signal-reversal/{n_classes}-sig-rev.pil")

INFO - Exported PIL to ..\artifacts\signal-reversal\3-sig-rev.pil


## LTA

In [2]:
N = 3
circ = DSDCircuit(fuel_multiplier=2, new_toehold_generation=True)
ext = circ.add_global("s", 2) 

inp = circ.new_signal("x", domain_len=LTA_PAPER_DOMAIN_LEN["input_signal"], dimension=N)
rev = circ.new_signal("y", domain_len=LTA_PAPER_DOMAIN_LEN["reversed_signal"], dimension=N)
out = circ.new_signal("z", domain_len=LTA_PAPER_DOMAIN_LEN["restored_signal"], dimension=N)
lta_func = [
    SignalReversal(inp, rev, reversible_design=False),
    PairwiseAnnihilation(rev, annihilator_conc=4),
    SignalRestoration(rev, out)
]

circ.add_modules(lta_func, add_reporting=True)

circ.export_PIL(standard_conc=50, output_file=f"../artifacts/lta-circuit/{N}-LTA-default.pil")

INFO - Exported PIL to ..\artifacts\lta-circuit\3-LTA-default.pil


In [ ]:
compile_LTA_circuit()